# **Задание 1 (10 баллов)**

Обучите языковую модель с помощью GRPO на вот этом датасете - notbadai/python_functions_reasoning Он чем-то похож на датасет из семинара, но вместо математических задач нужно написать код на питоне. Измените system prompt под эту задачу. Напишите аналогичные reward функции - одна на проверку формата (markdown wrappers python...  вокруг кода в конце), и на проверку что код работает (можете использовать import ast; ast.parse(string) для проверки) Если модель сначала не может отвечать в нужно формате, то сделайте цикл SFT обучения на небольшом куске датасета. GRPO в колабе может работать долго, поэтому не старайтесь пропустить весь датасет. Но обучайте хотя бы на 100 промптах (по несколько генераций на каждый промпт). Оцените разницу в качестве на небольшом сабсете, прогнав изначальную модель, модель после SFT (если она есть) и GRPO модель. В качестве метрики используйте reward функции. Можете взять любую языковую модель любого размера. Можете дообучать всю модель или же только адаптера поверх квантизированной модели.

# Импорт библиотек

In [1]:
!pip install datasets trl

In [2]:
!pip install bitsandbytes

In [3]:
from datasets import load_dataset
from trl import GRPOConfig, GRPOTrainer, SFTConfig, SFTTrainer, DPOTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from tqdm.notebook import tqdm
import pandas as pd
import torch
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
import random
from sklearn.model_selection import train_test_split
import re
import ast

# Датасет и инициализация модели

In [4]:
model_name = "Qwen/Qwen2-0.5B-Instruct"
device = 'cuda'

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [5]:
system_promt = """ Your task is to write a Python function.
Before providing the code, explain your approach step-by-step.
Put the final asnwer at the end preceeding by four hash signes like this: "#### *answer*.
Also, you have to do the markdown: "```your compiable python code ```""
"""

def chatml_format(example):
    messages = [{"role": "system", "content": system_promt},
                {"role": "user", "content": example['original_prompt']},
                {"role": "assistant", "content": str(example['reasoning']) + "\n####\n" + str(example['answer'])}]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return {"text": prompt}

In [6]:
pfr_dataset = load_dataset("notbadai/python_functions_reasoning", split="train")

# у меня там какая-то проблема возникла с именем переменных, так что я так сделала
pfr_dataset = pfr_dataset.rename_column("prompt", "original_prompt")

In [7]:
len(pfr_dataset)

206299

In [66]:
# возьмем 1000 строк для SFT, 100 строк для grpo и 100 строк для теста

test_size = 100
supervised_size = 1000
grpo_size = 100

random_seed=18
indices = list(range(len(pfr_dataset)))

put_aside, test_indices = train_test_split(indices, test_size=test_size,
                                           random_state=random_seed)

put_aside_2, grpo_indices = train_test_split(put_aside,
                                                    test_size=grpo_size,
                                                    random_state=random_seed)

put_aside_3, supervised_indices = train_test_split(put_aside_2,
                                               test_size=supervised_size,
                                               random_state=random_seed)

test_set = pfr_dataset.select(test_indices)
supervised_set = pfr_dataset.select(supervised_indices)
grpo_set = pfr_dataset.select(grpo_indices)

# grpo_set = grpo_set.map(chatml_format)
# test_set = test_set.map(chatml_format)
supervised_set = supervised_set.map(chatml_format)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [9]:
supervised_set[0]

{'original_prompt': '```python\ndef path_to_origin(path: str) -> bool:\n    """Write a function to determine if a given path string crosses itself at any point. The path string consists of characters \'N\', \'S\', \'E\', and \'W\', representing movements north, south, east, and west, respectively. You start at the origin (0, 0) on a 2D plane and walk according to the path specified by the string. The function should return `True` if the path crosses itself at any point, and `False` otherwise.\n    """\n```',
 'reasoning': "To solve this problem, we need to determine if a given path string causes the path to cross itself at any point. The path consists of movements in four directions: 'N' (North), 'S' (South), 'E' (East), and 'W' (West). We start at the origin (0, 0) on a 2D plane and follow the path specified by the string. Our goal is to detect if the path intersects itself at any point during the traversal.\n\nHere is a step-by-step plan to implement the function:\n\n1. **Initializat

# Обучение модели SFT

In [10]:
training_args = SFTConfig(
    max_length=2000,
    label_names=["labels"],
    report_to="none",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    logging_steps=10)

trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=supervised_set,
    args=training_args)

trainer.train()

Converting train dataset to ChatML:   0%|          | 0/1000 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Step,Training Loss
10,0.895800
20,0.627300
30,0.673900
40,0.676800
50,0.601200
60,0.629500
70,0.593900
80,0.569900
90,0.604200
100,0.626100


TrainOutput(global_step=750, training_loss=0.36078781239191693, metrics={'train_runtime': 1874.0039, 'train_samples_per_second': 1.601, 'train_steps_per_second': 0.4, 'total_flos': 3382697626843392.0, 'train_loss': 0.36078781239191693})

In [11]:
trainer.save_model('Qwen/Qwen2-0.5B-Instruct-pyfunc-sft')

# Обучение модели GRPO

## Подготовка

In [67]:
model_name = "Qwen/Qwen2-0.5B-Instruct-pyfunc-sft"
device = 'cuda'

In [68]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

In [69]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16
)
model = model.to(device)

In [70]:
system_prompt = """Your task is to write a Python function.
Before providing the code, explain your approach step-by-step.
Put the final asnwer at the end preceeding by four hash signes like this: "#### *answer*.
Also, you have to do the markdown: "```your compiable python code ```</s>"
"""

def chatml_format(example):
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": example['original_prompt']}]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    label = example['answer'] + "<|im_end|>\n"

    return {"prompt": prompt,
            "labels": label}

In [71]:
grpo_set = grpo_set.map(chatml_format)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

## Reward-функции

In [72]:
def markdown_python_wrapper_func(completions, **kwargs):
    pattern = re.compile(r'```python.*?```', re.DOTALL)
    return [1.0 if pattern.search(response) else 0.0 for response in completions]

def answer_working_func(completions, **kwargs):
    rewards = []
    pattern = re.compile(r'```python.*?```', re.DOTALL)

    for response in completions:
        code_blocks = pattern.findall(response)
        if not code_blocks:
            rewards.append(0.0)
            continue

        try:
            ast.parse(code_blocks[0].strip())
            rewards.append(1.0)
        except (IndentationError, SyntaxError):
            rewards.append(0.0)

    return rewards

In [73]:
batch = grpo_set[2:20]

input_ids = tokenizer.batch_encode_plus(batch['prompt'], return_tensors='pt', padding=True)

output = model.generate(input_ids['input_ids'].to(device),
                        attention_mask=input_ids['attention_mask'].to(device),
                        max_new_tokens=2000, do_sample=True,
                        temperature=1.5,
                        pad_token_id=tokenizer.eos_token_id)

In [74]:
completions = tokenizer.batch_decode(output, skip_special_tokens=False)
print(completions[0])

<|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|>

In [75]:
# проверяем работу функций
markdown_python_wrapper_func(completions)

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]

In [76]:
answer_working_func(completions)

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

Код пока не компилируется, но у нас все впереди

## Обучение

In [77]:
peft_config = LoraConfig(
    r=32,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules="all-linear"
)
training_args = GRPOConfig(output_dir="Qwen/Qwen2-0.5B-Instruct-math-sft-grpo",
                           logging_steps=10,
                           report_to="none",
                           num_generations=8,
                           num_train_epochs=1,
                           temperature=1.5,
                           label_names=["labels"])
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[answer_working_func, markdown_python_wrapper_func],
    args=training_args,
    train_dataset=grpo_set,
    peft_config=peft_config

)

In [78]:
trainer.train()

Step,Training Loss
10,0.018300
20,-0.000000
30,0.009800
40,0.010000
50,0.014900
60,0.017000
70,0.009900
80,0.002700
90,0.012600
100,0.004200


TrainOutput(global_step=100, training_loss=0.009939607020351104, metrics={'train_runtime': 2108.4204, 'train_samples_per_second': 0.047, 'train_steps_per_second': 0.047, 'total_flos': 0.0, 'train_loss': 0.009939607020351104})

Взяла очень мало в grpo, потому что компьютер устал(

In [ ]:
trainer.save_model('Qwen/Qwen2-0.5B-Instruct-pyfunc-grpo')

# Тестирование

In [79]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B-Instruct-pyfunc-grpo")
tokenizer.pad_token = tokenizer.eos_token
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"

OSError: Qwen/Qwen2-0.5B-Instruct-pyfunc-grpo is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
model_ref = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B-Instruct-math-sft",
                                                 device_map='cuda',
                                                 torch_dtype=torch.bfloat16)

model = PeftModel.from_pretrained(model_ref,
                                  "Qwen/Qwen2-0.5B-Instruct-pyfunc-grpo")

In [ ]:
device = model_ref.device
response_ref = []
dataset.set_format("pandas")
batch_size = 2
for i in tqdm(range(0, 100, batch_size)):
    batch = dataset[i:i+batch_size]
    input_ids = tokenizer.batch_encode_plus(batch['prompt'].tolist(), return_tensors='pt', padding=True)

    output = model_ref.generate(
        input_ids['input_ids'].to(device), attention_mask=input_ids['attention_mask'].to(device),
        max_new_tokens=200, do_sample=False, pad_token_id=tokenizer.eos_token_id #**gen_kwargs
    )[:,input_ids['input_ids'].shape[-1]:]
    response_ref.extend(tokenizer.batch_decode(output, skip_special_tokens=True))

In [ ]:
response = []

batch_size = 2
for i in tqdm(range(0, 100, batch_size)):
    batch = dataset[i:i+batch_size]
    input_ids = tokenizer.batch_encode_plus(batch['prompt'].tolist(), return_tensors='pt', padding=True)

    output = model.generate(
        input_ids['input_ids'].to(device), attention_mask=input_ids['attention_mask'].to(device),
        max_new_tokens=200, do_sample=False, pad_token_id=tokenizer.eos_token_id #**gen_kwargs
    )[:,input_ids['input_ids'].shape[-1]:]
    response.extend(tokenizer.batch_decode(output, skip_special_tokens=True))

In [ ]:
result_df = pd.DataFrame({'prompt': dataset['prompt'][:100],
                          'answer': dataset['answer'][:100],
                          'response (before)': response_ref,
                          'response (after)': response})

In [ ]:
def parse_answer(text):
    m = re.search('#### ?([\d.\-,]+)', text)
    if m is not None:
        return m.group(1)
    else:
        return ""

In [ ]:
result_df['answer_before'] = result_df["response (before)"].apply(parse_answer)
result_df['answer_after'] = result_df["response (after)"].apply(parse_answer)
result_df['answer_correct'] = result_df["answer"].apply(parse_answer)

In [ ]:
result_df[result_df['answer_before'] != result_df['answer_after']][['prompt',
                                                                    'answer_before',
                                                                    'answer_after',
                                                                    'answer_correct']]

In [ ]:
(result_df['answer_before'] == result_df['answer_correct']).sum()/len(result_df)

In [ ]:
(result_df['answer_after'] == result_df['answer_correct']).sum()/len(result_df)